In [1]:
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import numpy as np

In [2]:
!pip install gdown
!gdown 1FMVcFM78XZE1KE1rIkGBpCdcdI58S1LB
!unzip /content/custom_captions_dataset.zip
!rm -rf /content/custom_captions_dataset.zip

Streaming output truncated to the last 5000 lines.
  inflating: custom_captions_dataset/train/train_3367.jpg  
  inflating: custom_captions_dataset/train/train_3368.jpg  
  inflating: custom_captions_dataset/train/train_3370.jpg  
  inflating: custom_captions_dataset/train/train_3372.jpg  
  inflating: custom_captions_dataset/train/train_3373.jpg  
  inflating: custom_captions_dataset/train/train_3383.jpg  
  inflating: custom_captions_dataset/train/train_3392.jpg  
  inflating: custom_captions_dataset/train/train_3412.jpg  
  inflating: custom_captions_dataset/train/train_3413.jpg  
  inflating: custom_captions_dataset/train/train_3417.jpg  
  inflating: custom_captions_dataset/train/train_3422.jpg  
  inflating: custom_captions_dataset/train/train_3431.jpg  
  inflating: custom_captions_dataset/train/train_3432.jpg  
  inflating: custom_captions_dataset/train/train_3435.jpg  
  inflating: custom_captions_dataset/train/train_3442.jpg  
  inflating: custom_captions_dataset/train/train_

In [3]:
import os

os.environ["KERAS_BACKEND"] = "tensorflow"

import re
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import tensorflow as tf
import keras
from keras import layers
from keras.applications import efficientnet
from keras.layers import TextVectorization

keras.utils.set_random_seed(111)

In [4]:
import re
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import tensorflow as tf

#Path to the images
# IMAGES_PATH = "/content/DL_A2_subset_dataset"
# IMAGES_PATH = "/content/Flicker8k_Dataset"
IMAGES_PATH = "/content/custom_captions_dataset"

# Desired image dimensions
IMAGE_SIZE = (299, 299)

# Vocabulary size
# VOCAB_SIZE = 10000

# Fixed length allowed for any sequence
SEQ_LENGTH = 500

# Dimension for the image embeddings and token embeddings
EMBED_DIM = 512

# Per-layer units in the feed-forward network
FF_DIM = 512

# Other training parameters
#BATCH_SIZE = 64
BATCH_SIZE = 6
# EPOCHS = 30
EPOCHS=30
AUTOTUNE = tf.data.AUTOTUNE

PREPROCESSING

In [5]:


def load_captions_data(path,filename):
    """Loads captions (text) data and maps them to corresponding images.

    Args:
        filename: Path to the CSV file containing caption data.

    Returns:
        caption_mapping: Dictionary mapping image names and the corresponding captions
        text_data: List containing all the available captions
    """

    # Load the CSV file containing caption data
    captions_df = pd.read_csv(filename)

    caption_mapping = {}
    text_data = []

    for index, row in captions_df.iterrows():
        # Get the filename and caption from the DataFrame
        img_name = os.path.join(IMAGES_PATH, path, row['filename'])
        caption = row['caption']

        # We will remove captions that are either too short or too long
        tokens = caption.strip().split()

        if len(tokens) < 5 or len(tokens) > SEQ_LENGTH:
            continue

        # Add start and end tokens to each caption
        caption = "<start> " + caption.strip() + " <end>"
        text_data.append(caption)

        if img_name in caption_mapping:
            caption_mapping[img_name].append(caption)
        else:
            caption_mapping[img_name] = [caption]

    return captions_df, caption_mapping, text_data

def train_val_split(caption_data, train_size=0.8, shuffle=True):
    """Split the captioning dataset into train and validation sets.

    Args:
        caption_data (dict): Dictionary containing the mapped caption data
        train_size (float): Fraction of all the full dataset to use as training data
        shuffle (bool): Whether to shuffle the dataset before splitting

    Returns:
        Traning and validation datasets as two separated dicts
    """

    # 1. Get the list of all image names
    all_images = list(caption_data.keys())
    print("First few entries of all_images:")
    for img_name in all_images[:5]:  # Print the first 5 entries
        print(img_name)

    # 2. Shuffle if necessary
    if shuffle:
        np.random.shuffle(all_images)

    # 3. Split into training and validation sets
    train_size = int(len(caption_data) * train_size)
    print("train_size ",train_size)

    training_data = {
        img_name: caption_data[img_name] for img_name in all_images[:train_size]
    }
    validation_data = {
        img_name: caption_data[img_name] for img_name in all_images[train_size:]
    }

    # 4. Return the splits
    return training_data, validation_data


# Load the dataset
# captions_mapping, text_data = load_captions_data("Flickr8k.token.txt")
train_df, captions_mapping1, text_data1 = load_captions_data(f"{IMAGES_PATH}/train/","/content/custom_captions_dataset/train.csv")
val_df, captions_mapping2, text_data2 = load_captions_data(f"{IMAGES_PATH}/val/","/content/custom_captions_dataset/val.csv")
test_df, captions_mapping3, text_data3 = load_captions_data(f"{IMAGES_PATH}/test/","/content/custom_captions_dataset/test.csv")


# Combine caption mappings
captions_mapping = {**captions_mapping1, **captions_mapping2, **captions_mapping3}

# Concatenate text data
text_data = text_data1 + text_data2 + text_data3


# print("Entries from captions_mapping:")
# for idx, (img_name, captions) in enumerate(captions_mapping.items()):
#     print(f"Image: {img_name}")
#     print(f"Captions: {captions}")
#     print()

# # Print a few entries from text_data
# print("Entries from text_data:")
# for idx, caption in enumerate(text_data):
#     print(f"Caption {idx + 1}: {caption}")
#     print()



# Split the dataset into training and validation sets
# train_data, valid_data = train_val_split(captions_mapping)

# Initialize an empty training data dictionary
train_data = {}

# Iterate over the items in captions_mapping1
for img_name, captions in captions_mapping1.items():
    # Add the image name and its first caption to the training data
    train_data[img_name] = captions


# Initialize an empty training data dictionary
val_data = {}

# Iterate over the items in captions_mapping1
for img_name, captions in captions_mapping2.items():
    # Add the image name and its first caption to the training data
    val_data[img_name] = captions

test_data = {}

for img_name, captions in captions_mapping3.items():
    # Add the image name and its first caption to the training data
    test_data[img_name] = captions

# print(train_data)
# print(valid_data)
print("Number of training samples: ", len(train_data))
print("Number of validation samples: ", len(val_data))
print("Number of testing samples: ", len(test_data))

Number of training samples:  5715
Number of validation samples:  946
Number of testing samples:  928


In [6]:
import torch
from collections import Counter
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import numpy as np

class Vocabulary:
    def __init__(self):
        self.word2idx = {}
        self.idx2word = {}
        self.idx = 0

    def add_word(self, word):
        if word not in self.word2idx:
            self.word2idx[word] = self.idx
            self.idx2word[self.idx] = word
            self.idx += 1

    def __call__(self, word):
        if word not in self.word2idx:
            return self.word2idx['<unk>']
        return self.word2idx[word]

    def __len__(self):
        return len(self.word2idx)

def build_vocab(data, threshold=5):
    counter = Counter()
    for img_captions in data.values():
        for caption in img_captions:
            tokens = caption.lower().split()
            counter.update(tokens)

    words = [word for word, count in counter.items() if count >= threshold]

    vocab = Vocabulary()
    vocab.add_word('<pad>')
    vocab.add_word('<start>')
    vocab.add_word('<end>')
    vocab.add_word('<unk>')

    for word in words:
        vocab.add_word(word)

    return vocab

class CustomDataset(Dataset):
    def __init__(self, data, vocab, transform=None):
        self.data = data
        self.vocab = vocab
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_name = list(self.data.keys())[idx]
        captions = self.data[img_name]

        # Convert captions to list of lists of word indices
        caption_indices = [[self.vocab('<start>')] + [self.vocab(word) for word in caption.lower().split()] + [self.vocab('<end>')] for caption in captions]

        # Pad captions to the same length
        max_len = max(len(caption) for caption in caption_indices)
        padded_captions = [caption + [self.vocab('<pad>')] * (max_len - len(caption)) for caption in caption_indices]

        # Convert to tensor
        padded_captions = torch.tensor(padded_captions)

        # Load image
        image = Image.open(img_name).convert('RGB')
        if self.transform:
            image = self.transform(image)

        return image, padded_captions

def custom_collate_fn(batch):
    images, captions = zip(*batch)

    # Pad captions to the same length
    max_len = max(caption.size(1) for caption in captions)
    padded_captions = torch.zeros(len(captions), max_len).long()
    for i, caption in enumerate(captions):
        end = caption.size(1)
        padded_captions[i, :end] = caption.squeeze(0)
        # No need to manually pad the captions, as they're already padded during dataset initialization

    return torch.stack(images, dim=0), padded_captions


# Define transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# # Define your train_data and val_data
# train_data = {...}  # Your train_data in the format (img_path, captions)
# val_data = {...}    # Your val_data in the format (img_path, captions)

# Build vocabulary
vocab = build_vocab(train_data)

# Create datasets
train_dataset = CustomDataset(train_data, vocab, transform=transform)
val_dataset = CustomDataset(val_data, vocab, transform=transform)

# Define data loaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, collate_fn=custom_collate_fn)
val_loader = DataLoader(val_dataset, batch_size=64, collate_fn=custom_collate_fn)


MODEL CREATION

In [7]:
import torch
import torch.nn as nn
import torchvision.models as models
import torch.nn.functional as F

class EncoderCNN(nn.Module):
    def __init__(self, embed_size):
        super(EncoderCNN, self).__init__()
        resnet = models.resnet50(pretrained=True)
        for param in resnet.parameters():
            param.requires_grad_(False)

        modules = list(resnet.children())[:-1]
        self.resnet = nn.Sequential(*modules)
        self.embed = nn.Linear(resnet.fc.in_features, embed_size)
        #batch normalization
        self.batch= nn.BatchNorm1d(embed_size,momentum = 0.01)
        #Weights initialization
        self.embed.weight.data.normal_(0., 0.02)
        self.embed.bias.data.fill_(0)

    def forward(self, images):
        features = self.resnet(images)
        features = features.view(features.size(0), -1)
        features = self.batch(self.embed(features))
        return features

class DecoderRNN(nn.Module):
    def __init__(self, embed_size, hidden_size, vocab_size, num_layers=1):
        super(DecoderRNN, self).__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        self.embed_size = embed_size
        self.drop_prob = 0.2
        self.vocabulary_size = vocab_size

        # Define LSTM
        self.lstm = nn.LSTM(self.embed_size, self.hidden_size, self.num_layers, batch_first=True)
        self.dropout = nn.Dropout(self.drop_prob)
        self.embed = nn.Embedding(vocab_size, embed_size)
        self.linear = nn.Linear(hidden_size, vocab_size)

        # Weight initialization
        self.embed.weight.data.uniform_(-0.1, 0.1)
        self.linear.weight.data.uniform_(-0.1, 0.1)
        self.linear.bias.data.fill_(0)

    def forward(self, features, captions):
        #generating embedings from captures labels
        embeddings = self.embed(torch.tensor(captions))
        #Concatenate captions embedidings and images features in one dimension array
        features = features.unsqueeze(1)
        embeddings = torch.cat((features, embeddings[:, :-1,:]), dim=1)

        #Pack in sequences to create several batches with sequence length vocabulary size
        #packed = torch.nn.utils.rnn.pack_padded_sequence(embeddings, self.vocabulary_size,batch_first= True)
        #LSTM return hidden states and output of LSTM layers (score telling how near we are from finding the right word sequence)
        hiddens, c = self.lstm(embeddings)

        #Regression that feed to the next LSTM cell and contains the previous state
        outputs = self.linear(hiddens)
        #outputs = F.softmax(outputs)

        return outputs

    def sample(self, inputs, states=None, max_len=20):
        " accepts pre-processed image tensor (inputs) and returns predicted sentence (list of tensor ids of length max_len) "
        sampled_ids = []
        #inputs = inputs.unsqueeze(1)
        for i in range(max_len):
            #LSTM cell h, c
            hidden, states = self.lstm(inputs,states)
            outputs = self.linear(hidden.squeeze(1))
            #arg max probability per output in LSTM cell
            _, predicted = outputs.max(1)
            sampled_ids.append(predicted)
            #Update Hidden state with new output to next LSTM cell
            #How to tell if the index is word-vector index?
            inputs = self.embed(predicted)
            inputs = inputs.unsqueeze(1)

        sampled_ids = torch.stack(sampled_ids, 1)                # sampled_ids: (batch_size, max_seq_length)
        sampled_ids = list(sampled_ids.cpu().numpy()[0])
        sampled_ids = [int(i) for i in sampled_ids]
        return  sampled_ids


MODEL TRAINING

In [12]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import numpy as np
from tqdm import tqdm

print(len(train_loader))
# Define your encoder and decoder
embed_size = 512
hidden_size = 512
vocab_size = len(vocab) #10000  # Your vocabulary size

encoder = EncoderCNN(embed_size)
print(embed_size, hidden_size, vocab_size)
decoder = DecoderRNN(embed_size, hidden_size, vocab_size)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(list(encoder.parameters()) + list(decoder.parameters()), lr=0.001)

# Training loop
num_epochs = 1
best_val_loss = float('inf')  # Initialize with a very large value

for epoch in range(num_epochs):
    encoder.train()
    decoder.train()
    total_loss = 0
    total_correct = 0
    total_samples = 0

    progress_bar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}', unit='batch')
    for images, captions in progress_bar:
        optimizer.zero_grad()

        features = encoder(images)
        outputs = decoder(features, captions)

        targets = captions[:, :].contiguous().view(-1)
        outputs = outputs.view(-1, vocab_size)
        loss = criterion(outputs, targets)

        _, predicted = torch.max(outputs, 1)
        correct = (predicted == targets).sum().item()
        total_correct += correct
        total_samples += targets.size(0)

        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        progress_bar.set_postfix({'loss': total_loss / (len(train_loader) + 1e-12), 'accuracy': total_correct / (total_samples + 1e-12)})

    accuracy = total_correct / total_samples
    print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {total_loss / len(train_loader):.4f}, Accuracy: {accuracy:.4f}")

    encoder.eval()
    decoder.eval()
    total_val_loss = 0
    total_val_correct = 0
    total_val_samples = 0

    with torch.no_grad():
        for images, captions in val_loader:
            features = encoder(images)
            outputs = decoder(features, captions)
            targets = captions[:, :].contiguous().view(-1)
            loss = criterion(outputs.view(-1, vocab_size), targets)
            total_val_loss += loss.item()

    #         _, predicted = torch.max(outputs, 1)
    #         correct = (predicted == targets).sum().item()
    #         total_val_correct += correct
    #         total_val_samples += targets.size(0)

    # val_accuracy = total_val_correct / total_val_samples
    print(f"Epoch [{epoch+1}/{num_epochs}], Validation Loss: {total_val_loss / len(val_loader):.4f}")
    # Save the model if validation loss decreases
    if total_val_loss < best_val_loss:
        best_val_loss = total_val_loss
        torch.save({
            'encoder_state_dict': encoder.state_dict(),
            'decoder_state_dict': decoder.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'epoch': epoch,
            'train_loss': total_loss / len(train_loader),
            'val_loss': total_val_loss / len(val_loader)
        }, f'checkpoint_epoch_{epoch+1}.pt')

90
512 512 3297


Epoch 1/1:   0%|          | 0/90 [00:00<?, ?batch/s]<ipython-input-7-07a605e17445>:69: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  embeddings = self.embed(torch.tensor(captions))
Epoch 1/1: 100%|██████████| 90/90 [34:04<00:00, 22.72s/batch, loss=3.13, accuracy=0.564]


Epoch [1/1], Train Loss: 3.1296, Accuracy: 0.5641
Epoch [1/1], Validation Loss: 2.3183


In [14]:
!pip install nltk
!pip install git+https://github.com/ruotianluo/cider.git

  Cloning https://github.com/ruotianluo/cider.git to /tmp/pip-req-build-9wq77rzl
  Running command git clone --filter=blob:none --quiet https://github.com/ruotianluo/cider.git /tmp/pip-req-build-9wq77rzl
  Resolved https://github.com/ruotianluo/cider.git to commit dbb3960165d86202ed3c417b412a000fc8e717f3
ERROR: git+https://github.com/ruotianluo/cider.git does not appear to be a Python project: neither 'setup.py' nor 'pyproject.toml' found.


MODEL EVALUATION

In [15]:
test_dataset = CustomDataset(test_data, vocab, transform=transform)

# Define data loaders
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=True, collate_fn=custom_collate_fn)


In [20]:
import matplotlib.pyplot as plt

# Assuming test_loader is your DataLoader for the test dataset
max_caption_length = 128
# Set model to evaluation mode
encoder.eval()
decoder.eval()

# Define a function to generate captions for test images
def generate_caption(image):
    # Extract features using the encoder
    with torch.no_grad():
        features = encoder(image.unsqueeze(0))  # Add an extra dimension for batch size

        # Initialize the caption with the start token
        caption = [vocab('<start>')]

        # Generate the caption word by word using the decoder
        for _ in range(max_caption_length):
            # Convert the current caption to tensor
            current_caption = torch.tensor(caption, dtype=torch.long).unsqueeze(0)

            # Predict the next word
            outputs = decoder(features, current_caption)

            # Get the predicted word index
            predicted_index = torch.argmax(outputs, dim=2)

            # If the predicted word is the end token, stop generating the caption
            if predicted_index.item() == vocab('<end>'):
                break

            # Add the predicted word index to the caption
            caption.append(predicted_index.item())

    # Convert the generated caption indices back to words
    generated_caption = [vocab.idx2word[idx] for idx in caption]

    # Return the generated caption as a string
    return ' '.join(generated_caption)

# Iterate through the test dataset and generate captions for each image
for images, _ in test_loader:
    for image in images:
        # Generate caption for the current image
        generated_caption = generate_caption(image)

        # Convert the image tensor to numpy array and transpose the dimensions
        image_np = image.permute(1, 2, 0).numpy()

        # Display the image
        plt.imshow(image_np)
        plt.axis('off')
        plt.show()

        # Print the generated caption
        print(f"Generated Caption: {generated_caption}")


In [20]:
import nltk
# from cider import Cider
from nltk.translate.bleu_score import corpus_bleu
from nltk.translate.bleu_score import SmoothingFunction
from nltk.translate.bleu_score import sentence_bleu
# from nltk.translate.bleu_score import sentence_ngram_precisions
# from nltk.translate.rouge_score import rouge_l

# Initialize the CIDEr scorer
# cider_scorer = Cider()

# Set up nltk
nltk.download('punkt')
nltk.download('wordnet')

# Function to calculate BLEU score for a single caption
def calculate_bleu(reference, candidate):
    return sentence_bleu([reference], candidate, smoothing_function=SmoothingFunction().method1)

# Function to calculate ROUGE-L score for a single caption
# def calculate_rouge_l(reference, candidate):
#     return rouge_l([candidate], [reference])

# Initialize lists to store reference captions and generated captions
references = []
candidates = []

# Iterate through the test dataset and generate captions for each image
for images, captions in test_loader:
    for caption in captions:
        # Generate caption for the current image
        generated_caption = generate_caption(images)
        # Convert the image tensor to numpy array and transpose the dimensions
        image_np = image.permute(1, 2, 0).numpy()

        # Display the image
        plt.imshow(image_np)
        plt.axis('off')
        plt.show()

        # Print the generated caption
        print(f"Generated Caption: {generated_caption}")
        # Append the reference and generated captions
        references.append([caption.numpy().tolist()])
        candidates.append(generated_caption.split())

# Calculate BLEU score
bleu_scores = corpus_bleu(references, candidates)

# Calculate ROUGE-L score
# rouge_l_scores = [calculate_rouge_l(reference[0], candidate) for reference, candidate in zip(references, candidates)]

# Calculate CIDEr score
# cider_scores = cider_scorer.compute_score(references, candidates)

# Print evaluation metrics
print(f"BLEU Score: {bleu_scores}")
# print(f"ROUGE-L Score: {sum(rouge_l_scores) / len(rouge_l_scores)}")
# print(f"CIDEr Score: {cider_scores[0]}")